In [1]:
from __future__ import annotations
import os, re, requests
from typing import Optional, Dict, Any, List, Sequence, Annotated, TypedDict, Literal
from datetime import datetime, timedelta
from dateutil.parser import parse as dtparse
from pydantic import BaseModel

# LangChain / LangGraph
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, BaseMessage
from langchain.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, AnyMessage
from langgraph.prebuilt import ToolNode

# 환경 변수
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
KAMIS_CERT_KEY = os.environ.get("KAMIS_API_KEY")
KAMIS_CERT_ID = os.environ.get("KAMIS_CERT_ID")

# KAMIS API URL
KAMIS_URL = "http://www.kamis.or.kr/service/price/xml.do?action=dailyPriceByCategoryList"

# ===========================================
# 핵심 기능 1: 기본 정보 제공 (LLM 판단용 참고 자료)
# ===========================================

# 지역 코드 정보
REGION_INFO = {
    "1101": "서울", "2100": "부산", "2200": "대구", "2300": "인천", 
    "2401": "광주", "2501": "대전", "2601": "울산", "3111": "수원", "3911": "제주"
}

# 카테고리 정보
CATEGORY_INFO = {
    "100": "식량작물 (쌀, 밀, 보리, 콩, 옥수수 등 곡물류)",
    "200": "채소류 (배추, 무, 양파, 당근, 시금치, 상추 등)", 
    "300": "특용작물 (버섯, 인삼, 약용작물 등)",
    "400": "과일류 (사과, 배, 포도, 딸기, 바나나, 오렌지 등)",
    "500": "축산물 (소고기, 돼지고기, 닭고기, 계란, 우유 등)",
    "600": "수산물 (생선, 새우, 오징어, 명태, 고등어 등)"
}

# 소매/도매 정보
PRODUCT_CLS_INFO = {
    "01": "소매 (마트, 소매점에서 소비자에게 판매되는 가격)",
    "02": "도매 (가락시장, 공판장 등에서 거래되는 가격)"
}

@tool("kamis_param_infer")
def kamis_param_infer_tool(query: str) -> Dict[str, Any]:
    """
    사용자 자연어 쿼리 분석을 위한 기본 정보 제공
    LLM이 모든 파라미터를 직접 판단하도록 안내
    """
    today = datetime.now().date().strftime("%Y-%m-%d")
    
    guide = f"""
사용자 요청을 분석하여 KAMIS API 파라미터를 설정하세요.

=== 오늘 날짜 ===
오늘은 {today} 입니다.

=== 설정할 파라미터 ===

1. 카테고리 코드 (p_item_category_code):
{chr(10).join([f'   - {code}: {desc}' for code, desc in CATEGORY_INFO.items()])}

2. 소매/도매 구분 (p_product_cls_code):
{chr(10).join([f'   - {code}: {desc}' for code, desc in PRODUCT_CLS_INFO.items()])}

3. 지역 코드 (p_country_code) - 선택사항:
{chr(10).join([f'   - {code}: {name}' for code, name in REGION_INFO.items()])}

4. 조회 날짜 (p_regday) - 선택사항:
   - YYYY-MM-DD 형식으로 입력 (예: {today})
   - 오늘: {today}
   - 어제: {(datetime.now().date() - timedelta(days=1)).strftime("%Y-%m-%d")}
   - 미입력시 최신 가능한 날짜로 자동 조회

5. kg 환산 여부 (p_convert_kg_yn):
   - Y: kg 기준으로 환산하여 표시
   - N: 원래 단위 그대로 표시

사용자 요청: "{query}"

위 정보를 바탕으로 kamis_daily_price_by_category 도구를 적절한 파라미터와 함께 호출하세요.
"""
    
    return {
        "guide": guide,
        "today": today,
        "available_categories": CATEGORY_INFO,
        "available_regions": REGION_INFO,
        "product_cls_options": PRODUCT_CLS_INFO,
        "_note": "LLM이 사용자 요청을 분석하여 적절한 파라미터로 kamis_daily_price_by_category를 호출해야 합니다."
    }

# ===========================================
# 핵심 기능 2: KAMIS API 호출
# ===========================================

class KamisParams(BaseModel):
    p_cert_key: str
    p_cert_id: str
    p_returntype: Literal["json"] = "json"
    p_product_cls_code: Literal["01", "02"] = "02"
    p_item_category_code: Literal["100", "200", "300", "400", "500", "600"] = "100"
    p_country_code: Optional[str] = None
    p_regday: Optional[str] = None
    p_convert_kg_yn: Literal["Y", "N"] = "N"

def call_kamis_api(params: KamisParams) -> Dict[str, Any]:
    """KAMIS API 호출"""
    query_params = params.model_dump(exclude_none=True)
    query_params["action"] = "dailyPriceByCategoryList"
    
    response = requests.get(KAMIS_URL, params=query_params)
    return response.json()

@tool("kamis_daily_price_by_category", args_schema=KamisParams)
def kamis_tool(**kwargs) -> Dict[str, Any]:
    """KAMIS 일별 가격 정보 조회"""
    if not kwargs.get("p_cert_key"): 
        kwargs["p_cert_key"] = KAMIS_CERT_KEY
    if not kwargs.get("p_cert_id"): 
        kwargs["p_cert_id"] = KAMIS_CERT_ID
    
    params = KamisParams(**kwargs)
    result = call_kamis_api(params)
    return result

# ===========================================
# 핵심 기능 3: LLM Agent 구성
# ===========================================

class AgentState(TypedDict):
    messages: Annotated[Sequence[AnyMessage], add_messages]

def build_kamis_agent():
    """KAMIS 조회 에이전트 생성"""
    
    # LLM 설정
    llm = ChatOpenAI(model="gpt-5-mini", temperature=0)
    tools = [kamis_param_infer_tool, kamis_tool]
    llm_with_tools = llm.bind_tools(tools)
    
    # 그래프 생성
    graph = StateGraph(AgentState)
    
    def agent_node(state: AgentState):
        response = llm_with_tools.invoke(state["messages"])
        return {"messages": [response]}
    
    tool_node = ToolNode(tools)
    
    graph.add_node("agent", agent_node)
    graph.add_node("tools", tool_node)
    graph.set_entry_point("agent")
    
    def should_continue(state: AgentState):
        last_message = state["messages"][-1]
        if isinstance(last_message, AIMessage) and last_message.tool_calls:
            return "tools"
        return END
    
    graph.add_conditional_edges("agent", should_continue, {"tools": "tools", END: END})
    graph.add_edge("tools", "agent")
    
    return graph.compile()

# ===========================================
# 사용 예시
# ===========================================

SYSTEM_PROMPT = """당신은 KAMIS 농산물 가격 조회 도우미입니다.

사용자가 가격 조회를 요청하면 다음 단계를 따르세요:

1. 먼저 kamis_param_infer 도구를 호출하여 기본 정보와 가이드를 받으세요.
2. 사용자 질의를 분석하여 다음을 판단하세요:
   - 카테고리: 어떤 농산물 분류에 해당하는가?
   - 소매/도매: 소비자 가격(소매) vs 시장 가격(도매)?
   - 지역: 특정 지역이 언급되었는가?
   - 날짜: 언제 기준 가격인가? (오늘 날짜 정보 참고)
   - 단위: kg 기준 환산이 필요한가?

3. 판단한 결과로 kamis_daily_price_by_category 도구를 호출하세요.
4. 결과를 사용자에게 친화적으로 설명하세요.

중요 규칙: 
- 모든 파라미터는 사용자 요청과 컨텍스트를 바탕으로 직접 판단하세요.
- 애매한 경우 기본 옵션을 선택하세요.
- 날짜가 명시되지 않으면 최신 데이터를 조회하도록 하세요.
- 데이터가 "-"인 경우 반드시 다른 시점의 데이터를 확인하여 가장 가까운 유효한 가격을 찾아서 답변하고, 다른 이유에 대해서 설명하세요."""

def query_kamis(user_query: str):
    """KAMIS 조회 실행"""
    app = build_kamis_agent()
    
    messages = [
        SystemMessage(content=SYSTEM_PROMPT),
        HumanMessage(content=user_query)
    ]
    
    result = app.invoke({"messages": messages})
    return result["messages"][-1].content

# 테스트 예시
if __name__ == "__main__":
    result = query_kamis("어제 부산 소매 사과 가격을 알려줘.")
    print(result)

요청하신 어제(2025-09-18) 부산 소매 사과 가격 조회 결과입니다. (출처: KAMIS, 소매 기준)

주요 품목(단위: 10개)
- 홍로(상품): 24,525원 (2025-09-18) — 전일(09/17) 23,525원
- 홍로(중품): 16,600원 (2025-09-18) — 전일(09/17) 20,325원
- 쓰가루(아오리, 중품): 09/18 데이터는 “-”로 제공되어 당일 가격 없음. 가장 가까운 유효값은 2025-09-04의 19,900원(10개)이며(1개월 전 표기: 19,697원), 당일 가격 부재로 인해 최신 유효값(09/04)을 안내드립니다.

참고
- 조회 대상: 소매(마트/소매점), 지역: 부산(코드 2100), 날짜: 2025-09-18
- 단위는 표기된 그대로(대부분 10개)입니다. kg 기준 환산을 원하시면 알려주시면 환산해서 다시 계산해 드립니다.
- 특정 품종이나 등급(예: 상품/중품)을 지정하시면 더 정확히 찾아드릴게요.

더 필요한 정보가 있으신가요? (도·소매 전환, kg 환산, 다른 지역/날짜 등)


In [2]:
from __future__ import annotations
import os, re, requests
from typing import Optional, Dict, Any, List, Sequence, Annotated, TypedDict, Literal
from datetime import datetime, timedelta
from dateutil.parser import parse as dtparse
from pydantic import BaseModel

# LangChain / LangGraph
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, BaseMessage
from langchain.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, AnyMessage
from langgraph.prebuilt import ToolNode

# 환경 변수
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
KAMIS_CERT_KEY = os.environ.get("KAMIS_API_KEY")
KAMIS_CERT_ID = os.environ.get("KAMIS_CERT_ID")

# KAMIS API URL
KAMIS_URL = "http://www.kamis.or.kr/service/price/xml.do?action=dailyPriceByCategoryList"

# ===========================================
# 핵심 기능 1: 기본 정보 제공 (LLM 판단용 참고 자료)
# ===========================================

# 소매가격 지역 코드 정보
RETAIL_REGION_INFO = {
    "1101": "서울", "2100": "부산", "2200": "대구", "2300": "인천", 
    "2401": "광주", "2501": "대전", "2601": "울산", "3111": "수원", 
    "3214": "강릉", "3211": "춘천", "3311": "청주", "3511": "전주", 
    "3711": "포항", "3911": "제주", "3113": "의정부", "3613": "순천", 
    "3714": "안동", "3814": "창원", "3145": "용인", "2701": "세종", 
    "3112": "성남", "3138": "고양", "3411": "천안", "3818": "김해"
}

# 도매가격 지역 코드 정보
WHOLESALE_REGION_INFO = {
    "1101": "서울", "2100": "부산", "2200": "대구", 
    "2401": "광주", "2501": "대전"
}

# 카테고리 정보
CATEGORY_INFO = {
    "100": "식량작물 (쌀, 밀, 보리, 콩, 옥수수 등 곡물류)",
    "200": "채소류 (배추, 무, 양파, 당근, 시금치, 상추 등)", 
    "300": "특용작물 (버섯, 인삼, 약용작물 등)",
    "400": "과일류 (사과, 배, 포도, 딸기, 바나나, 오렌지 등)",
    "500": "축산물 (소고기, 돼지고기, 닭고기, 계란, 우유 등)",
    "600": "수산물 (생선, 새우, 오징어, 명태, 고등어 등)"
}

# 소매/도매 정보
PRODUCT_CLS_INFO = {
    "01": "소매 (마트, 소매점에서 소비자에게 판매되는 가격)",
    "02": "도매 (가락시장, 공판장 등에서 거래되는 가격)"
}

@tool("kamis_param_infer")
def kamis_param_infer_tool(query: str) -> Dict[str, Any]:
    """
    사용자 자연어 쿼리 분석을 위한 기본 정보 제공
    LLM이 모든 파라미터를 직접 판단하도록 안내
    """
    today = datetime.now().date().strftime("%Y-%m-%d")
    
    guide = f"""
사용자 요청을 분석하여 KAMIS API 파라미터를 설정하세요.

=== 오늘 날짜 ===
오늘은 {today} 입니다.

=== 설정할 파라미터 ===

1. 카테고리 코드 (p_item_category_code):
{chr(10).join([f'   - {code}: {desc}' for code, desc in CATEGORY_INFO.items()])}

2. 소매/도매 구분 (p_product_cls_code):
{chr(10).join([f'   - {code}: {desc}' for code, desc in PRODUCT_CLS_INFO.items()])}

3. 지역 코드 (p_country_code) - 선택사항:
   - None 또는 생략: 전체지역 (기본값)
   
   소매가격 선택가능 지역:
{chr(10).join([f'     - {code}: {name}' for code, name in RETAIL_REGION_INFO.items()])}
   
   도매가격 선택가능 지역:
{chr(10).join([f'     - {code}: {name}' for code, name in WHOLESALE_REGION_INFO.items()])}
   
   주의: 
   - 전체지역 조회시 p_country_code를 None으로 설정하거나 생략하세요
   - 소매가격과 도매가격은 선택 가능한 지역이 다릅니다!

4. 조회 날짜 (p_regday) - 선택사항:
   - YYYY-MM-DD 형식으로 입력 (예: {today})
   - 오늘: {today}
   - 어제: {(datetime.now().date() - timedelta(days=1)).strftime("%Y-%m-%d")}
   - 미입력시 최신 가능한 날짜로 자동 조회

5. kg 환산 여부 (p_convert_kg_yn):
   - Y: kg 기준으로 환산하여 표시
   - N: 원래 단위 그대로 표시

사용자 요청: "{query}"

위 정보를 바탕으로 kamis_daily_price_by_category 도구를 적절한 파라미터와 함께 호출하세요.
소매/도매 구분에 따라 해당하는 지역만 선택할 수 있음에 주의하세요.
"""
    
    return {
        "guide": guide,
        "today": today,
        "available_categories": CATEGORY_INFO,
        "retail_regions": RETAIL_REGION_INFO,
        "wholesale_regions": WHOLESALE_REGION_INFO,
        "product_cls_options": PRODUCT_CLS_INFO,
        "_note": "LLM이 사용자 요청을 분석하여 적절한 파라미터로 kamis_daily_price_by_category를 호출해야 합니다. 소매/도매에 따라 지역 선택이 제한됩니다."
    }

# ===========================================
# 핵심 기능 2: KAMIS API 호출
# ===========================================
class KamisParams(BaseModel):
    p_cert_key: str
    p_cert_id: str
    p_returntype: Literal["json"] = "json"
    p_product_cls_code: Literal["01", "02"] = "02"
    p_item_category_code: Literal["100", "200", "300", "400", "500", "600"] = "100"
    p_country_code: Optional[str] = None
    p_regday: Optional[str] = None
    p_convert_kg_yn: Literal["Y", "N"] = "N"

def call_kamis_api(params: KamisParams) -> Dict[str, Any]:
    """KAMIS API 호출"""
    query_params = params.model_dump(exclude_none=True)
    query_params["action"] = "dailyPriceByCategoryList"
    
    response = requests.get(KAMIS_URL, params=query_params)
    return response.json()

@tool("kamis_daily_price_by_category", args_schema=KamisParams)
def kamis_tool(**kwargs) -> Dict[str, Any]:
    """KAMIS 일별 가격 정보 조회"""
    if not kwargs.get("p_cert_key"): 
        kwargs["p_cert_key"] = KAMIS_CERT_KEY
    if not kwargs.get("p_cert_id"): 
        kwargs["p_cert_id"] = KAMIS_CERT_ID
    
    params = KamisParams(**kwargs)
    result = call_kamis_api(params)
    return result

# ===========================================
# 핵심 기능 3: LLM Agent 구성
# ===========================================

class AgentState(TypedDict):
    messages: Annotated[Sequence[AnyMessage], add_messages]

def build_kamis_agent():
    """KAMIS 조회 에이전트 생성"""
    
    # LLM 설정
    llm = ChatOpenAI(model="gpt-5-mini", temperature=0)
    tools = [kamis_param_infer_tool, kamis_tool]
    llm_with_tools = llm.bind_tools(tools)
    
    # 그래프 생성
    graph = StateGraph(AgentState)
    
    def agent_node(state: AgentState):
        response = llm_with_tools.invoke(state["messages"])
        return {"messages": [response]}
    
    tool_node = ToolNode(tools)
    
    graph.add_node("agent", agent_node)
    graph.add_node("tools", tool_node)
    graph.set_entry_point("agent")
    
    def should_continue(state: AgentState):
        last_message = state["messages"][-1]
        if isinstance(last_message, AIMessage) and last_message.tool_calls:
            return "tools"
        return END
    
    graph.add_conditional_edges("agent", should_continue, {"tools": "tools", END: END})
    graph.add_edge("tools", "agent")
    
    return graph.compile()

# ===========================================
# 사용 예시
# ===========================================

SYSTEM_PROMPT = """당신은 KAMIS 농산물 가격 조회 도우미입니다.

사용자가 가격 조회를 요청하면 다음 단계를 따르세요:

1. 먼저 kamis_param_infer 도구를 호출하여 기본 정보와 가이드를 받으세요.
2. 사용자 질의를 분석하여 다음을 판단하세요:
   - 카테고리: 어떤 농산물 분류에 해당하는가?
   - 소매/도매: 소비자 가격(소매) vs 시장 가격(도매)?
   - 지역: 특정 지역이 언급되었는가? (소매/도매별 지역 제한 확인)
   - 날짜: 언제 기준 가격인가? (오늘 날짜 정보 참고)
   - 단위: kg 기준 환산이 필요한가?

3. 판단한 결과로 kamis_daily_price_by_category 도구를 호출하세요.
4. 결과를 사용자에게 친화적으로 설명하세요.

중요 규칙: 
- 모든 파라미터는 사용자 요청과 컨텍스트를 바탕으로 직접 판단하세요.
- 소매가격과 도매가격은 조회 가능한 지역이 다릅니다! 반드시 확인하세요.
- 애매한 경우 기본 옵션을 선택하세요.
- 날짜가 명시되지 않으면 최신 데이터를 조회하도록 하세요.
- 데이터가 "-"인 경우 반드시 다른 시점의 데이터를 확인하여 가장 가까운 유효한 가격을 찾아서 답변하고, 다른 이유에 대해서 설명하세요.
- 지역 코드가 해당 소매/도매에서 지원되지 않는 경우, 사용자에게 알리고 대안을 제시하세요."""

def query_kamis(user_query: str):
    """KAMIS 조회 실행"""
    app = build_kamis_agent()
    
    messages = [
        SystemMessage(content=SYSTEM_PROMPT),
        HumanMessage(content=user_query)
    ]
    
    result = app.invoke({"messages": messages})
    return result["messages"][-1].content

# 테스트 예시
if __name__ == "__main__":
    result = query_kamis("서울 마트에서 파는 배추 가격 알려줘")
    print(result)

알겠습니다 — 요청하신 대로 서울(마트, 소매)에서의 배추 가격을 KAMIS에 조회했습니다.

제가 사용한 조회 정보(기본값)
- 품목 분류: 채소류 (코드 200, 배추 포함)
- 소매/도매: 소매(마트) (코드 01)
- 지역: 서울 (지역코드 1101)
- 날짜: 미지정 → 최신 데이터로 조회
- 단위환산: 원래 단위(kg 환산 미적용)

현재 결과
- KAMIS 호출 결과 가격 데이터가 반환되지 않았습니다(응답에 가격 항목이 없음). 이는 DEMO 인증키 제한 또는 해당 일자(또는 최신 데이터)에 서울 소매 단위의 가격 데이터가 등록되어 있지 않기 때문일 수 있습니다.

다음 중 어떻게 진행할까요? 원하시는 옵션을 골라 알려주세요.
1) (권장) 가장 최근의 “유효한” 가격(데이터가 있는 가장 가까운 날짜)으로 재조회 — 단위는 기본(마트 표기 단위) 또는 kg로 환산(Y/N) 선택 가능  
2) 전국(전체지역) 평균 가격으로 조회  
3) 특정 날짜(예: 2025-09-18 등)를 지정해서 조회  
4) 지금은 괜찮음(추가 조회 원하지 않음)

단위 관련 안내: 마트 소매는 종종 '포기(개)' 단위로 표시됩니다. kg 기준 가격을 원하시면 kg 환산(Y)으로 조회해 드립니다.

어떤 옵션으로 진행할까요?
